In [1]:
from django.contrib.gis.gdal import DataSource
from helper_functions.LayerMapping2 import LayerMapping
import os
import shapefile
import re
from django.db import transaction

In [5]:
mapping = {'name': 'name', 'geom': 'POLYGON',
           'start_date': 'start_date', 'end_date': 'end_date', 'date_accuracy': 'date_acc',
           'orig_id': 'id'}

In [3]:
folder = '/mnt/acdh_resources/container/R_histogis_7663/mosaic/18_Österreich/austro-hungarian-empire_1910'

In [6]:
ns, created = SkosNamespace.objects.get_or_create(namespace='https://histogis.acdh.oeaw.ac.at/vocabs/administrative_levels', prefix='hg')
scheme, created = SkosConceptScheme.objects.get_or_create(dc_title='Administrative Levels HistoGis', namespace=ns)

In [11]:
lst_files = {}
lst_files_id = {}
src, created = Source.objects.get_or_create(name='Austria-Hungary 1910 (mosaic data)', original_url='http://www.censusmosaic.org/mosaic/tmp/download/austro-hungarian_empire_1910.zip')
for fn in os.listdir(folder):
    if fn.endswith('.shp'):
        print(fn)
        fn_short = fn.split('.')[0]
        #r = shapefile.Reader(os.path.join(folder, fn_short))
        lm = LayerMapping(TempSpatial, os.path.join(folder, fn), mapping, encoding='latin1')
        lm.save(verbose=True)
        adm = re.search('adm[0-9]+', fn_short)
        voc, created = SkosConcept.objects.get_or_create(pref_label=adm.group(0), namespace=ns)
        if created:
            voc.scheme.add(scheme)
        for obj_id in lm.saved_object_ids:
            print(obj_id)
            obj = TempSpatial.objects.get(pk=obj_id)
            obj.source = src
            obj.administrative_unit = voc
            obj.save()
            lst_files['{}|{}'.format(fn_short, obj.name)] = obj_id

austro-hungarian-empire_adm2_1910.shp
Saved: Theiss-Maros-Becken
Saved: Bosnien-Herzegowina
Saved: Istrien
Saved: Niederoesterreich
Saved: Fiume
Saved: Salzburg
Saved: Kroatien-Slawonien
Saved: Linkes Donauufer
Saved: Steiermark
Saved: Krain
Saved: Vorarlberg
Saved: Bukowina
Saved: Boehmen
Saved: Linkes Theissufer
Saved: Schlesien
Saved: Siebenbuergen
Saved: Goerz & Gradisca
Saved: Istrien
Saved: Oberoesterreich
Saved: Dalmatien
Saved: Rechtes Donauufer
Saved: Donau-Theiss-Becken
Saved: Kaernten
Saved: Tirol
Saved: Galizien
Saved: Rechtes Theissufer
Saved: Maehren
10368
10369
10370
10371
10372
10373
10374
10375
10376
10377
10378
10379
10380
10381
10382
10383
10384
10385
10386
10387
10388
10389
10390
10391
10392
10393
10394
austro-hungarian-empire_adm1_1910.shp
Saved: Österreich-Ungarn
10395
austro-hungarian-empire_adm3_1910.shp
Saved: Friedeck Stadt
Saved: Esztergom
Saved: Alsó-Fehér
Saved: Görz (Landbezirk)
Saved: Vas
Saved: Árva
Saved: Mies
Saved: Pribram
Saved: Vsestin
Saved: Voitsb

Saved: Böhmisch Leipa
Saved: Sokal
Saved: Mährisch Ostrau
Saved: Krakau Stadt
Saved: Pisek
Saved: Littau
Saved: Budapest Munizipalstadt
Saved: Bihac
Saved: Monfalcone
Saved: Pozsony
Saved: Drohobycz
Saved: Wolfsberg
Saved: Wadowice
Saved: Vrhgorac
Saved: Landskron
Saved: Krems
Saved: Marburg (Landbezrik)
Saved: Hohenstadt
Saved: Szekesfehervar
Saved: Szabolcs
Saved: Fiume
Saved: Kuttenberg
Saved: Prestitz
Saved: Selcan
Saved: Stadt Olmütz
Saved: Jász-Nagykun-Szolnok
Saved: Stadt Waidhofen an der Ybbs
Saved: Mödling
Saved: Zell am See
Saved: Treffen
Saved: Warasdin
Saved: Triest
Saved: Liptó
Saved: Komotau
Saved: Rumburg
Saved: Bacs-Bodrog
Saved: Limanowa
Saved: Wittingau
Saved: Smichow
Saved: Perast
Saved: Mosciska
Saved: Horodenka
Saved: Kaaden
Saved: Warasdin Stadt
Saved: Luditz
Saved: Curzola
Saved: Jaslo
Saved: Pilgram
Saved: Ungarisch Hradisch
Saved: Cilli Stadt
Saved: Neustadtl
Saved: Bruck an der Mur
Saved: Tulln
Saved: Podgórze
Saved: Spalato
Saved: Wels
Saved: Marburg Stadt
Sa

Saved: Liszki
Saved: Falkenau/Falknov
Saved: Neustraschitz/Nove Stras
Saved: Maehrisch Altstadt/Stare
Saved: Sziszkow/Sziszkov
Saved: Tyczyn
Saved: Gliniany/H?yiniany
Saved: Grulich/Kraliky
Saved: Pilzno
Saved: Chlumetz/Chlumec
Saved: Z?oczow/Zo?ocziw
Saved: Prag/Praha St.
Saved: Wojnicz
Saved: Petschau/Becov
Saved: Adlerkosteletz/Kostelec
Saved: Koenigliche Weinberge/Kr
Saved: Podgorze
Saved: Bochnia
Saved: Zbara?
Saved: Jechnitz/Jesenice
Saved: Zator
Saved: Jaegerndorf
Saved: Niepo?omice
Saved: Winniki/Wynnyky
Saved: Boehmisch Brod/cesky Bro
Saved: Tarnow
Saved: Zborow/Zboriw
Saved: Luditz/Szlutice
Saved: Senftenberg/Szamberk
Saved: O?wi?cim/Auschwitz
Saved: Brzostek
Saved: Dubiecko
Saved: Lemberg Umgebung/Lwow Ok
Saved: Eger/Cheb
Saved: Holitz/Holice
Saved: Wiesenberg/Viesenberk
Saved: Strzy?ow
Saved: Grodek Jagiello?ski/Horo
Saved: Unhost/Unhost
Saved: Smichow/Smichov
Saved: Nusle
Saved: Skawina
Saved: Wieliczka
Saved: Rakonitz/Rakovnik
Saved: Pardubitz/Pardubice
Saved: S?dowa-Wisz

Saved: Varanno
Saved: Frauenberg/Hluboka [Podh
Saved: Hrottowitz/Hrotovice
Saved: Gwo?dziec
Saved: Iglo
Saved: Maehrisch Budwitz/Moravs
Saved: Iglo [Neudorf; Novawes]
Saved: Netolitz/Netolice
Saved: Bojkowitz/Bojkovice
Saved: Hartmanitz/Hartmanice
Saved: Zastawna/Zastavna
Saved: Neubistritz/Nova Bystric
Saved: Datschitz/Dacice
Saved: Turoczszentmarton
Saved: Lemes [Lemeshani]
Saved: Klobouk/Klobouky
Saved: Gaya/Kyjov
Saved: Rozsahegy
Saved: Seelowitz/Szidlochovice
Saved: Goelniczbanya
Saved: Illava
Saved: Lischau/Lisov
Saved: Goelniczbanya [Goellnitz
Saved: Pohrlitz/Pohorelice
Saved: Ungarisch Ostra/Uhersky
Saved: Dobsina [Dobschau] Magis
Saved: Szobrancz
Saved: Prachatitz/Prachatice
Saved: Jamnitz/Jemnice
Saved: Maehrisch Kromau/Moravsk
Saved: Dolina/Dolyna
Saved: So?otwina/Solotwyna
Saved: Ungarisch Brod/Uhersky B
Saved: Winterberg/Vimperk
Saved: Garamvoelgy (Nandorvoelg
Saved: Perecseny [Precsin]
Saved: Skole
Saved: Kolomea/Ko?omyja
Saved: Alsovereczke
Saved: Wallern/Volary
Saved: B

Saved: Nagykaroly
Saved: Erd?d
Saved: Gyoengyoes
Saved: Salzburg
Saved: Gmunden
Saved: Szentendre [Sankt Andrae
Saved: Magyarovar [Ungarisch Al
Saved: Nagysomkut [Somcuta mare
Saved: Magyarlapos [Lapusu ungu
Saved: Thalgau
Saved: Nezsider [Neusiedl am Se
Saved: Weyer
Saved: Hallein
Saved: Gyoer [Raab] Munizipalst
Saved: Oradna [Rodnavechia; Alt
Saved: Oberburg/Gornji Grad
Saved: Miklosvar (Nagyajta)
Saved: Voelgyseg (Bonyhad)
Saved: Szeged [Szegedin] Munizi
Saved: Kisenyed (Vizakna [Salzb
Saved: Nagylak [Nagylacu]
Saved: Nagysink [Grossschenk; C
Saved: Fogaras Magistratsstadt
Saved: Radmannsdorf/Radovljica
Saved: Cilli/Celje St.
Saved: Magyarpecska
Saved: Karfreit/Caporetto/Kobar
Saved: Cembra
Saved: Cavalese (Cavalese [Gabl
Saved: Rohitsch/Rogatec
Saved: Ivanec/Ivanecz
Saved: Kaposvar
Saved: Varaszdin/Varasd [Warasd
Saved: Baja Munizipalstadt
Saved: Mariaradna
Saved: Szaszsebes [Muehlbach; S
Saved: Franz/Vransko
Saved: Vizakna [Salzburg; Okna]
Saved: Ujarad
Saved: Lugos
Saved: Toercsv

Saved: Kastelruth/Castelrotto
Saved: Alsolendva
Saved: Lengyeltoti [Lengyeltoti
Saved: Homorod (Okland)
Saved: Oroshaza
Saved: Schlanders
Saved: Luttenberg/Ljutomer
Saved: Dics?szentmarton
Saved: Dunafoeldvar (Paks)
Saved: Mahrenberg/Marenberg
Saved: Mez?kovacshaza
Saved: Lana
Saved: Abrudbanya [Abrudu] Magi
Saved: Ampezzo (Cortina dAmpezz
Saved: Rosegg/Roszak
Saved: Medgyes [Mediasch; Media
Saved: Kisjen? [Jino]
Saved: Verespatak [Rosia]
Saved: Kezdivasarhely Magistrat
Saved: Buchenstein/Livinallongo
Saved: Simontornya (Gyoenk)
Saved: Ferlach/Borovlje
Saved: Koezpont (Mako)
Saved: Nagyhalmagy
Saved: Segesvar
Saved: Marburg/Maribor
Saved: Koezpont (Szekszard)
Saved: Bleiburg/Pliberk
Saved: Nagykanizsa Magistratsst
Saved: Hodmez?vasarhely Munizip
Saved: Fassa (Vigo di Fassa/Wig
Saved: Hosszuaszo
Saved: Kronau/Kranjska Gora
Saved: Toevis [Tyius]
Saved: Fondo
Saved: Tarvis/Trbisz
Saved: Friedau/Ormosz
Saved: Pettau/Ptuj St.
Saved: Dombovar
Saved: Kalocsa
Saved: Bozen/Bolzano
Saved: Windis

Saved: Gracac
Saved: Travnik
Saved: Obbrovazzo/Obrovac
Saved: Srebrenica
Saved: Vlasenica
Saved: Visoko
Saved: Kistanje
Saved: Knin
Saved: Glamoc
Saved: Zaravecchia/Biograd
Saved: Sarajevo St.
Saved: Benkovac
Saved: Fojnica
Saved: Scardona/Skradin
Saved: Vrlika
Saved: Bugojno
Saved: Stretto/Tijesno
Saved: Rogatica
Saved: Sebenico/sibenik
Saved: Livno
Saved: Prozor
Saved: Drnis
Saved: Visegrad
Saved: Sarajevo
Saved: cajnice
Saved: Sinj
Saved: Konjica
Saved: Szupanjac
Saved: Spalato/Split
Saved: Traù/Trogir
Saved: Almissa/Omis
Saved: Foca
Saved: Mostar St.
Saved: Imotski
Saved: Brazza/Bra? (San Pietro/
Saved: Mostar
Saved: Vrhgorac
Saved: Lesina/Hvar
Saved: Ljubuski
Saved: Nevesinje
Saved: Cittavecchia/Stari Grad
Saved: Macarsca/Makarska
Saved: Gacko
Saved: Lissa/Vis
Saved: Metkovi?
Saved: Stolac
Saved: Curzola/Korcula
Saved: Blatta/Blato
Saved: Sabbioncello/Peljesac (O
Saved: Ljubinje
Saved: Stagno/Ston
Saved: Bileca [Bilek]
Saved: Ragusa/Dubrovnik
Saved: Perasto/Perast
Saved: Castelnuo

12080
12081
12082
12083
12084
12085
12086
12087
12088
12089
12090
12091
12092
12093
12094
12095
12096
12097
12098
12099
12100
12101
12102
12103
12104
12105
12106
12107
12108
12109
12110
12111
12112
12113
12114
12115
12116
12117
12118
12119
12120
12121
12122
12123
12124
12125
12126
12127
12128
12129
12130
12131
12132
12133
12134
12135
12136
12137
12138
12139
12140
12141
12142
12143
12144
12145
12146
12147
12148
12149
12150
12151
12152
12153
12154
12155
12156
12157
12158
12159
12160
12161
12162
12163
12164
12165
12166
12167
12168
12169
12170
12171
12172
12173
12174
12175
12176
12177
12178
12179
12180
12181
12182
12183
12184
12185
12186
12187
12188
12189
12190
12191
12192
12193
12194
12195
12196
12197
12198
12199
12200
12201
12202
12203
12204
12205
12206
12207
12208
12209
12210
12211
12212
12213
12214
12215
12216
12217
12218
12219
12220
12221
12222
12223
12224
12225
12226
12227
12228
12229
12230
12231
12232
12233
12234
12235
12236
12237
12238
12239
12240
12241
12242
12243
12244
12245
1224

In [8]:
lst_files

{'austro-hungarian-empire_adm1_1910|Österreich-Ungarn': 2034,
 'austro-hungarian-empire_adm2_1910|Boehmen': 2019,
 'austro-hungarian-empire_adm2_1910|Bosnien-Herzegowina': 2008,
 'austro-hungarian-empire_adm2_1910|Bukowina': 2018,
 'austro-hungarian-empire_adm2_1910|Dalmatien': 2026,
 'austro-hungarian-empire_adm2_1910|Donau-Theiss-Becken': 2028,
 'austro-hungarian-empire_adm2_1910|Fiume': 2011,
 'austro-hungarian-empire_adm2_1910|Galizien': 2031,
 'austro-hungarian-empire_adm2_1910|Goerz & Gradisca': 2023,
 'austro-hungarian-empire_adm2_1910|Istrien': 2024,
 'austro-hungarian-empire_adm2_1910|Kaernten': 2029,
 'austro-hungarian-empire_adm2_1910|Krain': 2016,
 'austro-hungarian-empire_adm2_1910|Kroatien-Slawonien': 2013,
 'austro-hungarian-empire_adm2_1910|Linkes Donauufer': 2014,
 'austro-hungarian-empire_adm2_1910|Linkes Theissufer': 2020,
 'austro-hungarian-empire_adm2_1910|Maehren': 2033,
 'austro-hungarian-empire_adm2_1910|Niederoesterreich': 2010,
 'austro-hungarian-empire_adm2_1

In [13]:
with transaction.atomic():    
    for fn in os.listdir(folder):
        if fn.endswith('.shp'):
            fn_short = fn.split('.')[0]
            r = shapefile.Reader(os.path.join(folder, fn_short), encoding="latin1")
            for rr in r.iterShapeRecords():
                parent = rr.record[6]
                idx = rr.record[0]
                print(parent)
                if len(parent) > 5:
                    try:
                        parent = parent.decode('latin1')
                    except:
                        pass
                    try:
                        parent2 = lst_files[parent]
                    except:
                        parent = parent.replace('ü', 'ue')
                        parent = parent.replace('ö', 'oe')
                        parent = parent.replace('ä', 'ae')
                    try:    
                        parent2 = lst_files[parent]
                    except:
                        print('key error: {}'.format(parent))
                        continue
                    t1 = TempSpatial.objects.get(pk=parent2)
                    t2 = TempSpatial.objects.get(orig_id=idx)
                    t2.part_of = t1
                    t2.save()

b'austro-hungarian-empire_adm1_1910|\xd6sterreich-Ungarn'
b'austro-hungarian-empire_adm1_1910|\xd6sterreich-Ungarn'
b'austro-hungarian-empire_adm1_1910|\xd6sterreich-Ungarn'
b'austro-hungarian-empire_adm1_1910|\xd6sterreich-Ungarn'
b'austro-hungarian-empire_adm1_1910|\xd6sterreich-Ungarn'
b'austro-hungarian-empire_adm1_1910|\xd6sterreich-Ungarn'
b'austro-hungarian-empire_adm1_1910|\xd6sterreich-Ungarn'
b'austro-hungarian-empire_adm1_1910|\xd6sterreich-Ungarn'
b'austro-hungarian-empire_adm1_1910|\xd6sterreich-Ungarn'
b'austro-hungarian-empire_adm1_1910|\xd6sterreich-Ungarn'
b'austro-hungarian-empire_adm1_1910|\xd6sterreich-Ungarn'
b'austro-hungarian-empire_adm1_1910|\xd6sterreich-Ungarn'
b'austro-hungarian-empire_adm1_1910|\xd6sterreich-Ungarn'
b'austro-hungarian-empire_adm1_1910|\xd6sterreich-Ungarn'
b'austro-hungarian-empire_adm1_1910|\xd6sterreich-Ungarn'
b'austro-hungarian-empire_adm1_1910|\xd6sterreich-Ungarn'
b'austro-hungarian-empire_adm1_1910|\xd6sterreich-Ungarn'
b'austro-hunga

austro-hungarian-empire_adm2_1910|Donau-Theiss-Becken
b'austro-hungarian-empire_adm2_1910|Nieder\xf6sterreich'
austro-hungarian-empire_adm2_1910|Steiermark
austro-hungarian-empire_adm2_1910|Steiermark
b'austro-hungarian-empire_adm2_1910|Nieder\xf6sterreich'
austro-hungarian-empire_adm2_1910|Rechtes Theissufer
austro-hungarian-empire_adm2_1910|Maehren
austro-hungarian-empire_adm2_1910|Galizien
austro-hungarian-empire_adm2_1910|Steiermark
austro-hungarian-empire_adm2_1910|Theiss-Maros-Becken
austro-hungarian-empire_adm2_1910|Bosnien-Herzegowina
b'austro-hungarian-empire_adm2_1910|B\xf6hmen'
austro-hungarian-empire_adm2_1910|Istrien
austro-hungarian-empire_adm2_1910|Galizien
austro-hungarian-empire_adm2_1910|Dalmatien
austro-hungarian-empire_adm2_1910|Rechtes Donauufer
austro-hungarian-empire_adm2_1910|Galizien
b'austro-hungarian-empire_adm2_1910|Siebenb\xfcrgen'
b'austro-hungarian-empire_adm2_1910|B\xf6hmen'
austro-hungarian-empire_adm2_1910|Rechtes Theissufer
austro-hungarian-empire_adm

b'austro-hungarian-empire_adm2_1910|Ober\xf6sterreich'
b'austro-hungarian-empire_adm2_1910|B\xf6hmen'
austro-hungarian-empire_adm2_1910|Rechtes Donauufer
b'austro-hungarian-empire_adm2_1910|B\xf6hmen'
b'austro-hungarian-empire_adm2_1910|B\xf6hmen'
b'austro-hungarian-empire_adm2_1910|B\xf6hmen'
b'austro-hungarian-empire_adm2_1910|Ober\xf6sterreich'
austro-hungarian-empire_adm2_1910|Steiermark
austro-hungarian-empire_adm2_1910|Donau-Theiss-Becken
austro-hungarian-empire_adm2_1910|Theiss-Maros-Becken
austro-hungarian-empire_adm2_1910|Galizien
austro-hungarian-empire_adm2_1910|Galizien
austro-hungarian-empire_adm2_1910|Maehren
b'austro-hungarian-empire_adm2_1910|B\xf6hmen'
austro-hungarian-empire_adm2_1910|Galizien
austro-hungarian-empire_adm2_1910|Steiermark
austro-hungarian-empire_adm2_1910|Galizien
austro-hungarian-empire_adm2_1910|Donau-Theiss-Becken
b'austro-hungarian-empire_adm2_1910|Nieder\xf6sterreich'
b'austro-hungarian-empire_adm2_1910|B\xf6hmen'
austro-hungarian-empire_adm2_1910

austro-hungarian-empire_adm2_1910|Tirol
b'austro-hungarian-empire_adm2_1910|B\xf6hmen'
b'austro-hungarian-empire_adm2_1910|Siebenb\xfcrgen'
austro-hungarian-empire_adm2_1910|Galizien
austro-hungarian-empire_adm2_1910|Kroatien-Slawonien
b'austro-hungarian-empire_adm2_1910|B\xf6hmen'
b'austro-hungarian-empire_adm2_1910|B\xf6hmen'
b'austro-hungarian-empire_adm2_1910|B\xf6hmen'
austro-hungarian-empire_adm2_1910|Maehren
b'austro-hungarian-empire_adm2_1910|B\xf6hmen'
austro-hungarian-empire_adm2_1910|Tirol
austro-hungarian-empire_adm2_1910|Tirol
austro-hungarian-empire_adm2_1910|Steiermark
b'austro-hungarian-empire_adm2_1910|Ober\xf6sterreich'
austro-hungarian-empire_adm2_1910|Steiermark
austro-hungarian-empire_adm2_1910|Galizien
austro-hungarian-empire_adm2_1910|Schlesien
b'austro-hungarian-empire_adm2_1910|Nieder\xf6sterreich'
austro-hungarian-empire_adm2_1910|Theiss-Maros-Becken
austro-hungarian-empire_adm2_1910|Galizien
austro-hungarian-empire_adm3_1910|Schluckenau
austro-hungarian-empir

austro-hungarian-empire_adm3_1910|Pilzno
austro-hungarian-empire_adm3_1910|Neubydszow
b'austro-hungarian-empire_adm3_1910|Zlocz\xf3w'
austro-hungarian-empire_adm3_1910|Prag Stadt
austro-hungarian-empire_adm3_1910|Brzesko
austro-hungarian-empire_adm3_1910|Karlsbad/Tepl
austro-hungarian-empire_adm3_1910|Reichenau
b'austro-hungarian-empire_adm3_1910|K\xf6nigliche Weinberge'
b'austro-hungarian-empire_adm3_1910|Podg\xf3rze'
austro-hungarian-empire_adm3_1910|Bochnia
austro-hungarian-empire_adm3_1910|Zbaraz
austro-hungarian-empire_adm3_1910|Podersam
austro-hungarian-empire_adm3_1910|Oswiecim
b'austro-hungarian-empire_adm3_1910|J\xe4gerndorf'
austro-hungarian-empire_adm3_1910|Bochnia
austro-hungarian-empire_adm3_1910|Lemberg
b'austro-hungarian-empire_adm3_1910|B\xf6hmisch Brod'
austro-hungarian-empire_adm3_1910|Tarnow
austro-hungarian-empire_adm3_1910|Zborow
austro-hungarian-empire_adm3_1910|Luditz
austro-hungarian-empire_adm3_1910|Senftenberg
austro-hungarian-empire_adm3_1910|Oswiecim
austro-

...

austro-hungarian-empire_adm3_1910|Szeged Munizipalstadt
b'austro-hungarian-empire_adm3_1910|Als\xf3-Feh\xe9r'
b'austro-hungarian-empire_adm3_1910|Csan\xe1d'
b'austro-hungarian-empire_adm3_1910|Nagy-K\xfck\xfcllo'
austro-hungarian-empire_adm3_1910|Fogaras
austro-hungarian-empire_adm3_1910|Radmannsdorf
austro-hungarian-empire_adm3_1910|Cilli Stadt
austro-hungarian-empire_adm3_1910|Arad
austro-hungarian-empire_adm3_1910|Tolmein
austro-hungarian-empire_adm3_1910|Cembra
austro-hungarian-empire_adm3_1910|Cavalese
austro-hungarian-empire_adm3_1910|Pettau
austro-hungarian-empire_adm3_1910|Warasdin
austro-hungarian-empire_adm3_1910|Somogy
austro-hungarian-empire_adm3_1910|Warasdin
austro-hungarian-empire_adm3_1910|Baja Munizipalstadt
austro-hungarian-empire_adm3_1910|Arad
austro-hungarian-empire_adm3_1910|Szeben
austro-hungarian-empire_adm3_1910|Cilli (Landbezirk)
b'austro-hungarian-empire_adm3_1910|Als\xf3-Feh\xe9r'
austro-hungarian-empire_adm3_1910|Temes
b'austro-hungarian-empire_adm3_1910|Kr

austro-hungarian-empire_adm3_1910|Murau
austro-hungarian-empire_adm3_1910|Szekesfehervar
austro-hungarian-empire_adm3_1910|Weiz
austro-hungarian-empire_adm3_1910|Vas
austro-hungarian-empire_adm3_1910|Innsbruck (Landbezirk)
austro-hungarian-empire_adm3_1910|Maros-Torda
austro-hungarian-empire_adm3_1910|Hartberg
austro-hungarian-empire_adm3_1910|Zell am See
austro-hungarian-empire_adm3_1910|Zell am See
austro-hungarian-empire_adm3_1910|Judenburg
austro-hungarian-empire_adm3_1910|Bihar
austro-hungarian-empire_adm3_1910|Kolozs
austro-hungarian-empire_adm3_1910|St. Johann
austro-hungarian-empire_adm3_1910|Vas
austro-hungarian-empire_adm3_1910|Judenburg
b'austro-hungarian-empire_adm3_1910|Veszpr\xe9m'
austro-hungarian-empire_adm3_1910|Bihar
austro-hungarian-empire_adm3_1910|Kolozs
austro-hungarian-empire_adm3_1910|Zell am See
austro-hungarian-empire_adm3_1910|Pest-Pilis-Solt-Kiskum
austro-hungarian-empire_adm3_1910|Pest-Pilis-Solt-Kiskum
austro-hungarian-empire_adm3_1910|Kolozs
austro-hungar

austro-hungarian-empire_adm3_1910|Zala
austro-hungarian-empire_adm3_1910|Somogy
austro-hungarian-empire_adm3_1910|Udvarhely
b'austro-hungarian-empire_adm3_1910|B\xe9k\xe9s'
austro-hungarian-empire_adm3_1910|Schlanders
austro-hungarian-empire_adm3_1910|Luttenberg
b'austro-hungarian-empire_adm3_1910|Kis-K\xfck\xfcllo'
austro-hungarian-empire_adm3_1910|Tolna
austro-hungarian-empire_adm3_1910|Windischgraz
b'austro-hungarian-empire_adm3_1910|Csan\xe1d'
austro-hungarian-empire_adm3_1910|Meran
b'austro-hungarian-empire_adm3_1910|Als\xf3-Feh\xe9r'
austro-hungarian-empire_adm3_1910|Ampezzo
austro-hungarian-empire_adm3_1910|Villach
b'austro-hungarian-empire_adm3_1910|Nagy-K\xfck\xfcllo'
austro-hungarian-empire_adm3_1910|Arad
b'austro-hungarian-empire_adm3_1910|Als\xf3-Feh\xe9r'
b'austro-hungarian-empire_adm3_1910|H\xe1romsz\xe9k'
austro-hungarian-empire_adm3_1910|Ampezzo
austro-hungarian-empire_adm3_1910|Tolna
austro-hungarian-empire_adm3_1910|Klagenfurt (Landbezirk)
b'austro-hungarian-empire_ad

austro-hungarian-empire_adm3_1910|Szeben
austro-hungarian-empire_adm3_1910|Loitsch
austro-hungarian-empire_adm3_1910|Adelsberg
austro-hungarian-empire_adm3_1910|Sesana
austro-hungarian-empire_adm3_1910|Zombor Munizipalstadt
austro-hungarian-empire_adm3_1910|Treffen
b'austro-hungarian-empire_adm3_1910|Toront\xe1l'
austro-hungarian-empire_adm3_1910|Rovereto (Landbezirk)
b'austro-hungarian-empire_adm3_1910|Bjelovar-Kri\x9eevci'
austro-hungarian-empire_adm3_1910|Temes
austro-hungarian-empire_adm3_1910|Monfalcone
austro-hungarian-empire_adm3_1910|Bacs-Bodrog
b'austro-hungarian-empire_adm3_1910|Bjelovar-Kri\x9eevci'
austro-hungarian-empire_adm3_1910|Monfalcone
austro-hungarian-empire_adm3_1910|Bacs-Bodrog
austro-hungarian-empire_adm3_1910|Adelsberg
austro-hungarian-empire_adm3_1910|Agram
b'austro-hungarian-empire_adm3_1910|Toront\xe1l'
austro-hungarian-empire_adm3_1910|Virovititz
austro-hungarian-empire_adm3_1910|Treffen
austro-hungarian-empire_adm3_1910|Temes
austro-hungarian-empire_adm3_19

austro-hungarian-empire_adm3_1910|Tuzla
austro-hungarian-empire_adm3_1910|Banjaluka
austro-hungarian-empire_adm3_1910|Tuzla
austro-hungarian-empire_adm3_1910|Tuzla
austro-hungarian-empire_adm3_1910|Lika-Krbava
austro-hungarian-empire_adm3_1910|Rab
austro-hungarian-empire_adm3_1910|Travnik
austro-hungarian-empire_adm3_1910|Bihac
austro-hungarian-empire_adm3_1910|Bihac
austro-hungarian-empire_adm3_1910|Travnik
austro-hungarian-empire_adm3_1910|Lika-Krbava
austro-hungarian-empire_adm3_1910|Travnik
austro-hungarian-empire_adm3_1910|Tuzla
austro-hungarian-empire_adm3_1910|Travnik
austro-hungarian-empire_adm3_1910|Lika-Krbava
austro-hungarian-empire_adm3_1910|Travnik
austro-hungarian-empire_adm3_1910|Benkovac
austro-hungarian-empire_adm3_1910|Tuzla
austro-hungarian-empire_adm3_1910|Tuzla
austro-hungarian-empire_adm3_1910|Sarajevo
austro-hungarian-empire_adm3_1910|Benkovac
austro-hungarian-empire_adm3_1910|Knin
austro-hungarian-empire_adm3_1910|Travnik
austro-hungarian-empire_adm3_1910|Rab
au